In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr

# Load data
data = pd.read_csv("TSLA.csv")

# Prepare data
data['Close_shifted'] = data['Close'].shift(2) # Target close 2 days ahead
data['Close_lag7'] = data['Close'].shift(7) # Lagged close data 7 days before
data['Open_lag7'] = data['Open'].shift(7) # Lagged open data 7 days before

# Drop missing values
data.dropna(inplace=True)

# Split data into features and target
X_close_only = data[['Close_lag7']]
X_close_open = data[['Close_lag7', 'Open_lag7']]
y = data['Close_shifted']

# Split data into training and testing sets
X_train_close_only, X_test_close_only, y_train, y_test = train_test_split(X_close_only, y, test_size=0.2, random_state=42)
X_train_close_open, X_test_close_open, _, _ = train_test_split(X_close_open, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_close_only_scaled = scaler.fit_transform(X_train_close_only)
X_test_close_only_scaled = scaler.transform(X_test_close_only)

X_train_close_open_scaled = scaler.fit_transform(X_train_close_open)
X_test_close_open_scaled = scaler.transform(X_test_close_open)

# Initialize models
mlp_close_only = MLPRegressor(random_state=42)
mlp_close_open = MLPRegressor(random_state=42)
knn_close_only = KNeighborsRegressor()
knn_close_open = KNeighborsRegressor()
dt_close_only = DecisionTreeRegressor(random_state=42)
dt_close_open = DecisionTreeRegressor(random_state=42)
rf_close_only = RandomForestRegressor(random_state=42)
rf_close_open = RandomForestRegressor(random_state=42)

# Train models
mlp_close_only.fit(X_train_close_only_scaled, y_train)
mlp_close_open.fit(X_train_close_open_scaled, y_train)
knn_close_only.fit(X_train_close_only_scaled, y_train)
knn_close_open.fit(X_train_close_open_scaled, y_train)
dt_close_only.fit(X_train_close_only, y_train)
dt_close_open.fit(X_train_close_open, y_train)
rf_close_only.fit(X_train_close_only, y_train)
rf_close_open.fit(X_train_close_open, y_train)

# Predictions
mlp_pred_close_only = mlp_close_only.predict(X_test_close_only_scaled)
mlp_pred_close_open = mlp_close_open.predict(X_test_close_open_scaled)
knn_pred_close_only = knn_close_only.predict(X_test_close_only_scaled)
knn_pred_close_open = knn_close_open.predict(X_test_close_open_scaled)
dt_pred_close_only = dt_close_only.predict(X_test_close_only)
dt_pred_close_open = dt_close_open.predict(X_test_close_open)
rf_pred_close_only = rf_close_only.predict(X_test_close_only)
rf_pred_close_open = rf_close_open.predict(X_test_close_open)

# Evaluate models
def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    vpearsonr = pearsonr(y_true, y_pred)
    return mse, mae, rmse, vpearsonr

mse_mlp_close_only, mae_mlp_close_only, rmse_mlp_close_only = evaluate(y_test, mlp_pred_close_only)
mse_mlp_close_open, mae_mlp_close_open, rmse_mlp_close_open = evaluate(y_test, mlp_pred_close_open)
mse_knn_close_only, mae_knn_close_only, rmse_knn_close_only = evaluate(y_test, knn_pred_close_only)
mse_knn_close_open, mae_knn_close_open, rmse_knn_close_open = evaluate(y_test, knn_pred_close_open)
mse_dt_close_only, mae_dt_close_only, rmse_dt_close_only = evaluate(y_test, dt_pred_close_only)
mse_dt_close_open, mae_dt_close_open, rmse_dt_close_open = evaluate(y_test, dt_pred_close_open)
mse_rf_close_only, mae_rf_close_only, rmse_rf_close_only = evaluate(y_test, rf_pred_close_only)
mse_rf_close_open, mae_rf_close_open, rmse_rf_close_open = evaluate(y_test, rf_pred_close_open)

# Print evaluation results
print("MLP (Close only) - MSE:", mse_mlp_close_only, "MAE:", mae_mlp_close_only, "RMSE:", rmse_mlp_close_only, "Pearsonr:", vpearsonr)
print("MLP (Close+Open) - MSE:", mse_mlp_close_open, "MAE:", mae_mlp_close_open, "RMSE:", rmse_mlp_close_open, "Pearsonr:", vpearsonr)
print("KNN (Close only) - MSE:", mse_knn_close_only, "MAE:", mae_knn_close_only, "RMSE:", rmse_knn_close_only)
print("KNN (Close+Open) - MSE:", mse_knn_close_open, "MAE:", mae_knn_close_open, "RMSE:", rmse_knn_close_open)
print("Decision Tree (Close only) - MSE:", mse_dt_close_only, "MAE:", mae_dt_close_only, "RMSE:", rmse_dt_close_only)
print("Decision Tree (Close+Open) - MSE:", mse_dt_close_open, "MAE:", mae_dt_close_open, "RMSE:", rmse_dt_close_open)
print("Random Forest (Close only) - MSE:", mse_rf_close_only, "MAE:", mae_rf_close_only, "RMSE:", rmse_rf_close_only)
print("Random Forest (Close+Open) - MSE:", mse_rf_close_open, "MAE:", mae_rf_close_open, "RMSE:", rmse_rf_close_open)

/home/aliif/Documents/kuliah/semester6/belajar-pandas/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/aliif/Documents/kuliah/semester6/belajar-pandas/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/aliif/Documents/kuliah/semester6/belajar-pandas/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


ValueError: too many values to unpack (expected 3)